In [0]:
dbutils.fs.ls("/mnt/AzureAccount")

[FileInfo(path='dbfs:/mnt/AzureAccount/DatabricksParquet/', name='DatabricksParquet/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/AzureAccount/Pyspark_With_Json/', name='Pyspark_With_Json/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/AzureAccount/SampleCSV.csv', name='SampleCSV.csv', size=176, modificationTime=1734508306000),
 FileInfo(path='dbfs:/mnt/AzureAccount/YearlyTopTen.csv', name='YearlyTopTen.csv', size=57828, modificationTime=1730564575000),
 FileInfo(path='dbfs:/mnt/AzureAccount/country_full.csv', name='country_full.csv', size=19732, modificationTime=1735229205000),
 FileInfo(path='dbfs:/mnt/AzureAccount/country_full_Upsert.csv', name='country_full_Upsert.csv', size=1270, modificationTime=1735229205000),
 FileInfo(path='dbfs:/mnt/AzureAccount/my_table/', name='my_table/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/AzureAccount/my_table1/', name='my_table1/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/AzureAccount/my_table_Updat

In [0]:
from delta.tables import DeltaTable

delta_table_path = "/mnt/AzureAccount/my_table_Updated"

DF_Basic = spark.read.csv(
    "/mnt/AzureAccount/country_full.csv", inferSchema=True, header=True, sep=","
).fillna('null').fillna(0)
DF_Basic_Updated = spark.createDataFrame(DF_Basic.select("ID","name","alpha-2","alpha-3","region").head(10))
display(DF_Basic_Updated)



ID,name,alpha-2,alpha-3,region
1,Afghanistan,AF,AFG,Asia
2,Åland Islands,AX,ALA,Europe
3,Albania,AL,ALB,Europe
4,Algeria,DZ,DZA,Africa
5,American Samoa,AS,ASM,Oceania
6,Andorra,AD,AND,Europe
7,Angola,AO,AGO,Africa
8,Anguilla,AI,AIA,Americas
9,Antarctica,AQ,ATA,null
10,Antigua and Barbuda,AG,ATG,Americas


In [0]:
DF_Basic_Updated.write.format("delta").save(delta_table_path)
Delta_Table_Updated = DeltaTable.createOrReplace(spark).location(delta_table_path)


In [0]:
from pyspark.sql.functions import col

Upsert_DF_Path = "/mnt/AzureAccount/country_full_Upsert.csv"

Upsert_DF = spark.read.csv(Upsert_DF_Path, inferSchema=True , header=True , sep="," ).fillna('n').fillna(0)
Upsert_DF_Updated = Upsert_DF.select([col("{}".format(str(i))) for i in Upsert_DF.columns])
display(Upsert_DF_Updated)

ID,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
1,United States Minor Outlying Islands,UM,UMI,581,ISO 3166-2:UM,Oceania,Micronesia,n,9,57,0
2,Uruguay,UY,URY,858,ISO 3166-2:UY,Americas,Latin America and the Caribbean,South America,19,419,5
5,Uzbekistan,UZ,UZB,860,ISO 3166-2:UZ,Asia,Central Asia,n,142,143,0
4,Vanuatu,VU,VUT,548,ISO 3166-2:VU,Oceania,Melanesia,n,9,54,0
51,Venezuela (Bolivarian Republic of),VE,VEN,862,ISO 3166-2:VE,Americas,Latin America and the Caribbean,South America,19,419,5
26,Viet Nam,VN,VNM,704,ISO 3166-2:VN,Asia,South-eastern Asia,n,142,35,0
47,Virgin Islands (British),VG,VGB,92,ISO 3166-2:VG,Americas,Latin America and the Caribbean,Caribbean,19,419,29
68,Virgin Islands (U.S.),VI,VIR,850,ISO 3166-2:VI,Americas,Latin America and the Caribbean,Caribbean,19,419,29
99,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,n,9,61,0
110,Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,n,2,15,0


In [0]:
from pyspark.sql import functions as F
Columns = Upsert_DF_Updated.columns
Column_List = r'F.concat_ws('
Pos = 0

for i in Columns:
    i = str(i)
    if Pos + 1 != len(Columns):
        
        Column_List = Column_List + r'F.lit(' + i + ' :)' + r' ,col("source.{' + str(Pos) + r'}")' + r", F.lit(,) ,"
    else:
        Column_List = Column_List  + r'F.lit(' + i + ' :)' + r' ,col("source.{' + str(Pos) + r'}"))'
    Pos = Pos + 1
# Column_List = Column_List + i + ":" + "{col(" + r'"{source.' + str(Pos) + r'}")}'
    
print(Column_List)

F.concat_ws(F.lit(ID :) ,col("source.{0}"), F.lit(,) ,F.lit(name :) ,col("source.{1}"), F.lit(,) ,F.lit(alpha-2 :) ,col("source.{2}"), F.lit(,) ,F.lit(alpha-3 :) ,col("source.{3}"), F.lit(,) ,F.lit(region :) ,col("source.{4}"))


In [0]:
from pyspark.sql.functions import col

def concat_cols(*list_cols):
    return '-'.join(dict({f'{str(i)}' , f'"source.{str(i)}"'} for i in list_cols))

print(concat_cols(Columns))
concat_cols = udf(concat_cols)
# def call(funcname):
#     dispatcher={'col':col , 'str':str}
#     return dispatcher[funcname]

# CallDispatcher={
#     'col':col ,
#      'str':str ,
#      'eval': eval}


['ID', 'name', 'alpha-2', 'alpha-3', 'region']


In [0]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from ast import literal_eval

delta_table = DeltaTable.forPath(spark,delta_table_path)

delta_table.alias("target") \
    .merge(Upsert_DF_Updated.alias("source"), \
       col("source.ID") == col("target.ID")) \
             .whenMatchedUpdate(set = { 
                str(i) : col("source.{}".format(str(i))) for i in Columns}
                                ) \
           .whenNotMatchedInsert(values = { 
              str(i) : col("source.{}".format(str(i))) for i in Columns}
                                 ).execute()
    

In [0]:
from delta.tables import DeltaTable

delta_table_path = "/mnt/AzureAccount/my_table_Updated"
delta_table = DeltaTable.forPath(spark,delta_table_path)
display(delta_table.toDF())

ID,name,alpha-2,alpha-3,region
1,United States Minor Outlying Islands,UM,UMI,Oceania
2,Uruguay,UY,URY,Americas
5,Uzbekistan,UZ,UZB,Asia
4,Vanuatu,VU,VUT,Oceania
51,Venezuela (Bolivarian Republic of),VE,VEN,Americas
26,Viet Nam,VN,VNM,Asia
47,Virgin Islands (British),VG,VGB,Americas
68,Virgin Islands (U.S.),VI,VIR,Americas
99,Wallis and Futuna,WF,WLF,Oceania
110,Western Sahara,EH,ESH,Africa


In [0]:
{literal_eval(Column_List.format(*Columns),{'__builtins__':None},CallDispatcher)}